In [1]:
# --------------------------------------------------------
# 🧠 Hyperparameter Optimization for CNN using CIFAR-10
# --------------------------------------------------------

!pip install keras-tuner --quiet

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
import numpy as np
import matplotlib.pyplot as plt

# 1️⃣ Load inbuilt CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize pixel values
X_train, X_test = X_train / 255.0, X_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()

# 2️⃣ Define model-building function for Keras Tuner
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(
        filters=hp.Int('conv_filters_1', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size', values=[3,5]),
        activation='relu',
        input_shape=(32,32,3)
    ))
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Conv2D(
        filters=hp.Int('conv_filters_2', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size2', values=[3,5]),
        activation='relu'
    ))
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Flatten())

    model.add(layers.Dense(
        units=hp.Int('dense_units', min_value=64, max_value=256, step=64),
        activation='relu'
    ))

    model.add(layers.Dropout(
        rate=hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)
    ))

    model.add(layers.Dense(10, activation='softmax'))  # 10 classes

    # Compile with variable learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# 3️⃣ Initialize Keras Tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,             # number of different combinations to try
    executions_per_trial=1,
    directory='cnn_tuning',
    project_name='cifar10_hpo'
)

# 4️⃣ Perform search
tuner.search(X_train, y_train, epochs=5, validation_split=0.2)

# 5️⃣ Get the best model
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparams = tuner.get_best_hyperparameters(1)[0]

print("\n🔹 Best Hyperparameters:")
for param, value in best_hyperparams.values.items():
    print(f"{param}: {value}")

# 6️⃣ Evaluate best model on test data
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=2)
print(f"\n✅ Test Accuracy: {test_acc*100:.2f}%")

# 7️⃣ Visualize model training history
history = best_model.fit(X_train, y_train, epochs=10, validation_split=0.2)



Trial 5 Complete [00h 06m 01s]
val_accuracy: 0.10670000314712524

Best val_accuracy So Far: 0.6294999718666077
Total elapsed time: 00h 38m 03s


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



🔹 Best Hyperparameters:
conv_filters_1: 32
kernel_size: 5
conv_filters_2: 32
kernel_size2: 3
dense_units: 192
dropout_rate: 0.2
learning_rate: 0.0004738715531002648
313/313 - 3s - 8ms/step - accuracy: 0.6344 - loss: 1.0416

✅ Test Accuracy: 63.44%
Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 32s 25ms/step - accuracy: 0.6721 - loss: 0.9336 - val_accuracy: 0.6629 - val_loss: 0.9624
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 31s 25ms/step - accuracy: 0.6966 - loss: 0.8683 - val_accuracy: 0.6717 - val_loss: 0.9442
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 39s 23ms/step - accuracy: 0.7168 - loss: 0.8061 - val_accuracy: 0.6745 - val_loss: 0.9407
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 30s 24ms/step - accuracy: 0.7349 - loss: 0.7527 - val_accuracy: 0.6831 - val_loss: 0.9181
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 30s 24ms/step - accuracy: 0.7488 - loss: 0.7095 - val_accuracy: 0.6807 - val_loss: 0.9233
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 30s 24ms/step - accuracy: 0.7632 - loss: 0.6740 -

In [3]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 12, 12, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 192)            │       221,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,930 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 704,960 (2.69 MB)

 Trainable params: 234,986 (917.91 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 469,974 (1.79 MB)